In [10]:
import json
import random
import re

In [22]:
with open('train_data.json', 'r', encoding='UTF-8') as file:
    data = json.load(file)

# validation data
# max participants: 10
# max turns: 20

# train data
# max participants: 10
# max turns: 20

In [23]:
# limit number of participants (max 2)
for i in range(len(data['data'])- 1, -1, -1):
    num_parts = data['data'][i]['header']['dialogueInfo']['numberOfParticipants']
    if num_parts > 2:
        del data['data'][i]

In [24]:
# remove special characters (except question mark)
special_characters_pattern = r"[^\w\s?]"
def preprocess_text(text):
    processed_text = re.sub(special_characters_pattern, '', text)
    return processed_text

In [25]:
# merge turns
# remove characters consisting only of consonants or vowels
# remove unnecessary columns
for i in range(len(data['data'])):
    del data['data'][i]['header']
    item = data['data'][i]
    merged_utter = {}
    for utter in item['body']:
        if utter['turnID'] not in merged_utter:
            merged_utter[utter['turnID']] = utter['utterance']
        else:
            merged_utter[utter['turnID']] += ' ' + utter['utterance']
    for j in range(len(item['body']) - 1, -1, -1):
        turn_ID = item['body'][j]['turnID']
        if turn_ID in merged_utter:
            item['body'][j]['utterance'] = merged_utter[turn_ID]
            item['body'][j]['utterance'] = preprocess_text(re.sub('([ㄱ-ㅎㅏ-ㅣ]+)','',item['body'][j]['utterance']))
            del merged_utter[turn_ID]
            del item['body'][j]['utteranceID']
            del item['body'][j]['turnID']
            del item['body'][j]['date']
            del item['body'][j]['time']
        else:
            del item['body'][j]


In [17]:
data['data'][0]

{'body': [{'participantID': 'P01', 'utterance': '나지금밥머거2시간걸어서 번화가찾았어 잉'},
  {'participantID': 'P02',
   'utterance': '헐  언넝호텔들가 엄청피건할첸데 나는인낫러요 나 두시출근이다 퀵으로한대서 두시까지오래 '},
  {'participantID': 'P01', 'utterance': '오좋겠네'},
  {'participantID': 'P02', 'utterance': '잘잣어??'},
  {'participantID': 'P01', 'utterance': '아니 자지도못했어 진짜피씨방에서30분? 너가좋아하는돈가쭈 머거'},
  {'participantID': 'P02', 'utterance': '잉 나도줘 내돈가쓰'},
  {'participantID': 'P01', 'utterance': '맛있어 고로케도존맛탱'},
  {'participantID': 'P02', 'utterance': '사진찍엇어???'},
  {'participantID': 'P01', 'utterance': ' 먼사진?'},
  {'participantID': 'P02', 'utterance': '돈따스'},
  {'participantID': 'P01', 'utterance': '안보내줫어? 엥보낸줄알았는대 시스템사진 이거'},
  {'participantID': 'P02', 'utterance': '하 '}]}

In [26]:
# generate subset of turns
data_copy = data['data'][:]
for item in data_copy:
    for i in range(3, len(item['body'])+1):
        subset = item['body'][:i]
        data['data'].append({'body': subset})

In [27]:
# randomly sample 500000 train data, 50000 test data
random_samples = random.sample(data['data'], 500000)

In [28]:
with open('train.json', 'w', encoding='UTF-8') as file:
    json.dump(random_samples, file, ensure_ascii=False, indent=4)